In [2]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist

In [ ]:

# GP
# Kernel
# Fit GP
# Optimize GP Hyper paramters
# Acquisition function
# Optimizing CNN hyper parameters

class GaussianProcess():
    def __init__(self, lengthscale : float, kernel_variance : float, noise_variance : float):
        self.lengthscale = torch.nn.Parameter(torch.tensor(float(lengthscale)), requires_grad=True )
        self.kernel_variance = torch.nn.Parameter(torch.tensor(float(kernel_variance)), requires_grad=True)
        self.noise_variance = torch.nn.Parameter(torch.tensor(float(noise_variance)), requires_grad=True)

        self.posterior_mean = None
        self.posterior_covariance = None

    def squared_exponential_kernel(self, x : torch.Tensor, y : torch.Tensor):
        x = x.squeeze(1).expand(x.size(0), y.size(0))
        y = y.squeeze(0).expand(x.size(0), y.size(0))
        sqdist = torch.pow(x - y, 2)
        covariance = self.kernel_variance * torch.exp(-sqdist / (2 * self.lengthscale**2))    
        return covariance
    
    def fit_posterior(self, xtrain : torch.Tensor, ytrain : torch.Tensor, xtest : torch.Tensor):
        N = len(xtrain)
        
        K = self.squared_exponential_kernel(xtrain, xtrain)
        Ks = self.squared_exponential_kernel(xtrain, xtest)
        Kss = self.squared_exponential_kernel(xtest, xtest)
        
        L = torch.linalg.cholesky(K + self.noise_variance * torch.eye(len(N)))
        _alpha_temp = torch.linalg.solve_triangular(L, ytrain,upper=False)
        _alpha = torch.linalg.solve_triangular(L.t(),_alpha_temp,upper=True)
        mu = Ks.t() @ _alpha
        
        v = torch.linalg.solve(L, Ks)
        covariance = Kss - v.t() @ v
        
        self.posterior_mean = mu
        self.posterior_covariance = covariance
        
        return mu, covariance
        
    def optimize():
        pass


In [ ]:
x = torch.tensor([1,2]).reshape(-1, 1)
y = torch.tensor([3,4]).reshape(-1, 1)

gp = GaussianProcess(1, 1)

(tensor([[3, 3],
         [4, 4]]),
 tensor([[1, 2],
         [1, 2]]))